# **Ejercicios de pair programming Módulo 2 - Sprint 2**
## **ETL III. Transformación II**
---

In [1]:
import requests
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL de Extracción I y Transformación I.

In [2]:
class Extraccion_limpieza():

    def __init__(self, lang, category, widget):
        self.lang = lang
        self.category = category
        self.widget = widget


    def extraccion_nacional(self, start_year, end_year, time_trunc):
        self.start_year = start_year
        self.end_year = end_year
        self.time_trunc = time_trunc

        df_vacio = pd.DataFrame()

        for año in range(start_year, end_year+1):
            url = f'https://apidatos.ree.es/{self.lang}/datos/{self.category}/{self.widget}?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc={time_trunc}'
            response = requests.get(url=url)
            print(f'Status code año:{año}: {response.status_code}')
            print(f'Reason año: {año}: {response.reason}')
            
            datos_electricidad = response.json()
            df_ren = pd.DataFrame(datos_electricidad['included'][0]['attributes']['values'])
            df_ren['type'] = datos_electricidad['included'][0]['type']
            df_no_ren = pd.DataFrame(datos_electricidad['included'][1]['attributes']['values'])
            df_no_ren['type'] = datos_electricidad['included'][1]['type']
            df_vacio = pd.concat([df_vacio, df_ren, df_no_ren], axis=0)

        df_vacio.to_csv(f'../datos/{self.widget}_nacional.csv')

        return df_vacio


    def extraccion_ccaa(self, start_year, end_year, time_trunc):
        self.start_year = start_year
        self.end_year = end_year
        self.time_trunc = time_trunc    
    
        cod_comunidades = {'Ceuta': 8744,
                        'Melilla': 8745,
                        'Andalucía': 4,
                        'Aragón': 5,
                        'Cantabria': 6,
                        'Castilla - La Mancha': 7,
                        'Castilla y León': 8,
                        'Cataluña': 9,
                        'País Vasco': 10,
                        'Principado de Asturias': 11,
                        'Comunidad de Madrid': 13,
                        'Comunidad Foral de Navarra': 14,
                        'Comunitat Valenciana': 15,
                        'Extremadura': 16,
                        'Galicia': 17,
                        'Illes Balears': 8743,
                        'Canarias': 8742,
                        'Región de Murcia': 21,
                        'La Rioja': 20}
        df_vacio = pd.DataFrame()

        for ccaa, geo_id in cod_comunidades.items():

            for año in range(start_year, end_year+1):
                url = f'https://apidatos.ree.es/{self.lang}/datos/{self.category}/{self.widget}?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc={time_trunc}&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={geo_id}'
                response = requests.get(url=url)
                
                datos_electricidad = response.json()
                try:
                    df_ren = pd.DataFrame(datos_electricidad['included'][0]['attributes']['values'])
                    df_ren['type'] = datos_electricidad['included'][0]['type']
                    df_ren['ccaa'] = ccaa
                    df_vacio = pd.concat([df_vacio, df_ren], axis=0)
                
                    df_no_ren = pd.DataFrame(datos_electricidad['included'][1]['attributes']['values'])
                    df_no_ren['type'] = datos_electricidad['included'][1]['type']
                    df_no_ren['ccaa'] = ccaa
                    df_vacio = pd.concat([df_vacio, df_no_ren], axis=0)
                except:
                    pass

        df_vacio.to_csv(f'../datos/{self.widget}_ccaa.csv')

        return df_vacio
    

    def limpieza(self, df, nombre_archivo, columnas_eliminar = []):
        self.df = df
        self.columnas_eliminar = columnas_eliminar

        df.drop(columns=columnas_eliminar, inplace=True)
        df[['percentage', 'value']] = df[['percentage', 'value']].apply(lambda dato: round(dato, 2))
        df['date'] = df['datetime'].str.split('T', expand=True).get(0)
        df['date'] = pd.to_datetime(df['date'])
        df.to_pickle(f'../datos/{nombre_archivo}.pkl')
        return df

In [3]:
renov_no_renov = Extraccion_limpieza('es', 'generacion', 'evolucion-renovable-no-renovable')
renov_no_renov

In [4]:
df_nacional = renov_no_renov.extraccion_nacional(2011, 2022, 'year')

Status code año:2011: 200
Reason año: 2011: OK
Status code año:2012: 200
Reason año: 2012: OK
Status code año:2013: 200
Reason año: 2013: OK
Status code año:2014: 200
Reason año: 2014: OK
Status code año:2015: 200
Reason año: 2015: OK
Status code año:2016: 200
Reason año: 2016: OK
Status code año:2017: 200
Reason año: 2017: OK
Status code año:2018: 200
Reason año: 2018: OK
Status code año:2019: 200
Reason año: 2019: OK
Status code año:2020: 200
Reason año: 2020: OK
Status code año:2021: 200
Reason año: 2021: OK
Status code año:2022: 200
Reason año: 2022: OK


In [5]:
df_nacional.head()

,value,percentage,datetime,type
0,8.666705e+07,0.310241,2011-01-01T00:00:00.000+01:00,Renovable
0,1.926868e+08,0.689759,2011-01-01T00:00:00.000+01:00,No renovable
0,8.533872e+07,0.301424,2012-01-01T00:00:00.000+01:00,Renovable
0,1.977799e+08,0.698576,2012-01-01T00:00:00.000+01:00,No renovable
0,1.097577e+08,0.401622,2013-01-01T00:00:00.000+01:00,Renovable


In [6]:
df_ccaa = renov_no_renov.extraccion_ccaa(2011, 2022, 'year')

In [7]:
df_ccaa.head()

,value,percentage,datetime,type,ccaa
0,202972.893,1.0,2011-01-01T00:00:00.000+01:00,No renovable,Ceuta
0,212071.911,1.0,2012-01-01T00:00:00.000+01:00,No renovable,Ceuta
0,201964.935,1.0,2013-01-01T00:00:00.000+01:00,No renovable,Ceuta
0,212245.157,1.0,2014-01-01T00:00:00.000+01:00,No renovable,Ceuta
0,204023.090,1.0,2015-01-01T00:00:00.000+01:00,No renovable,Ceuta


In [8]:
df_nacional = renov_no_renov.limpieza(df_nacional, 'renovables_no_renovables_nacional')

In [9]:
df_nacional.head()

,value,percentage,datetime,type,date
0,8.666705e+07,0.31,2011-01-01T00:00:00.000+01:00,Renovable,2011-01-01
0,1.926868e+08,0.69,2011-01-01T00:00:00.000+01:00,No renovable,2011-01-01
0,8.533872e+07,0.30,2012-01-01T00:00:00.000+01:00,Renovable,2012-01-01
0,1.977799e+08,0.70,2012-01-01T00:00:00.000+01:00,No renovable,2012-01-01
0,1.097577e+08,0.40,2013-01-01T00:00:00.000+01:00,Renovable,2013-01-01


In [10]:
df_ccaa = renov_no_renov.limpieza(df_ccaa, 'renovables_no_renovables_ccaa')

In [11]:
df_ccaa.head()

,value,percentage,datetime,type,ccaa,date
0,202972.89,1.0,2011-01-01T00:00:00.000+01:00,No renovable,Ceuta,2011-01-01
0,212071.91,1.0,2012-01-01T00:00:00.000+01:00,No renovable,Ceuta,2012-01-01
0,201964.94,1.0,2013-01-01T00:00:00.000+01:00,No renovable,Ceuta,2013-01-01
0,212245.16,1.0,2014-01-01T00:00:00.000+01:00,No renovable,Ceuta,2014-01-01
0,204023.09,1.0,2015-01-01T00:00:00.000+01:00,No renovable,Ceuta,2015-01-01
